# 04 — Predicting Glomerular Position from Odor Responses

Glomeruli in the olfactory bulb are not randomly scattered — their
positions reflect the functional organization of olfactory processing.
In this notebook we ask: **can we predict *where* a glomerulus sits on
the bulb from *how* it responds to odorants?**

We will use this question to walk through the core ideas of regression
in scikit-learn, building up in three stages:

| Stage | Features | Targets | Goal |
|-------|----------|---------|------|
| 1 | 1 odorant | y position | Simple linear regression |
| 2 | 2 odorants | y position | Multiple regression |
| 3 | All odorants | x *and* y position | Multi-output regression |

**Prerequisites:** Basic Python and NumPy. No prior machine-learning
experience is assumed.

In [ ]:
%pip install -q git+https://github.com/CastroLab/glom-explorer.git

---
## 0. Setup and data preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glom_explorer as gx

### Loading the data

The dataset contains calcium-imaging responses from glomeruli on the
dorsal surface of the mouse olfactory bulb.  Each glomerulus was
stimulated with a panel of **59 odorants** at low concentration, and
its fluorescence response was recorded.  We also know the **spatial
coordinates** (x, y) of every glomerulus.

`load_clustered_variant()` returns two DataFrames (low- and
high-concentration).  We will work with the **low-concentration**
data.  We use the `nonresponders_dropped` variant, which excludes
glomeruli whose mean response across all odorants is near zero —
these contribute no usable signal and would add noise to our
regressions.

Columns are glomeruli and rows are odorants, so we transpose
to get the conventional *samples × features* layout.

In [ ]:
# Load odorant metadata and clustered response matrices
odorants = gx.load_odorants()
low, high = gx.load_clustered_variant('nonresponders_dropped')

# Odorant names (we will use these as feature labels)
odorant_names = odorants['Name'].values
print(f"Number of odorants: {len(odorant_names)}")
print(f"Number of glomeruli: {low.shape[1]}")

In [ ]:
# Build the feature matrix X (glomeruli × odorants)
# and target vectors for y-position and x-position.

X = low.values.T                          # shape: (n_glomeruli, 59)
y_pos = low.columns.get_level_values('y').astype(float).values
x_pos = low.columns.get_level_values('x').astype(float).values

print(f"Feature matrix X:  {X.shape}  (glomeruli × odorants)")
print(f"Target y_pos:      {y_pos.shape}")
print(f"Target x_pos:      {x_pos.shape}")

### Quick look at the data

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 3.5))

axes[0].hist(y_pos, bins=30, edgecolor='white')
axes[0].set(xlabel='y position', ylabel='Count', title='Distribution of y positions')

axes[1].hist(X.mean(axis=1), bins=30, edgecolor='white')
axes[1].set(xlabel='Mean response', ylabel='Count',
            title='Mean response per glomerulus')

sc = axes[2].scatter(x_pos, y_pos, c=X.mean(axis=1), s=8, cmap='viridis', alpha=0.7)
axes[2].set(xlabel='x position', ylabel='y position',
            title='Glomeruli colored by mean response')
plt.colorbar(sc, ax=axes[2], label='Mean ΔF/F')

plt.tight_layout()
plt.show()

---
## 1. Single feature, single target

The simplest regression question we can ask: **can a glomerulus's
response to one odorant predict its y position?**

This is *simple linear regression* — fitting a line:

$$\hat{y} = w \cdot x + b$$

where $x$ is the response to a single odorant, $w$ is the slope
(coefficient), and $b$ is the intercept.

### 1.1 Choose an odorant and filter to responders

Let's start with **butyric acid**, a carboxylic acid with a strong,
rancid-butter smell.  It is odorant index 1 in our panel.

Even after removing global non-responders, most glomeruli do not
respond to any *particular* odorant — olfactory receptors are
selective.  If we regress on all glomeruli, the scatter plot will be
dominated by a blob of zeros at every y position, drowning out the
real signal.  So for single-odorant regression we **filter to
glomeruli that actually respond** (response > 0) to the chosen
odorant.

In [ ]:
odorant_idx = 1
print(f"Odorant: {odorant_names[odorant_idx]}")

# Filter to glomeruli that respond to this odorant
resp_mask = X[:, odorant_idx] > 0
X1_all = X[resp_mask]
y1_pos = y_pos[resp_mask]
feature = X1_all[:, odorant_idx]

print(f"Responders: {resp_mask.sum()} / {len(resp_mask)} glomeruli")

plt.figure(figsize=(6, 4))
plt.scatter(feature, y1_pos, s=8, alpha=0.5)
plt.xlabel(f'Response to {odorant_names[odorant_idx]}')
plt.ylabel('y position')
plt.title('Can one odorant predict position?')
plt.show()

### 1.2 Train / test split

Before we fit a model, we need to **hold out** some data for
evaluation.  If we tested on the same data we trained on, we would not
know whether the model has truly learned a generalizable pattern or
just memorized the training examples.

`train_test_split` randomly partitions the data.  We use 80 % for
training and reserve 20 % for testing.

In [ ]:
from sklearn.model_selection import train_test_split

# Reshape to a column vector — sklearn expects 2-D input
X1 = feature.reshape(-1, 1)

X1_train, X1_test, y_train, y_test = train_test_split(
    X1, y1_pos, test_size=0.2, random_state=42
)

print(f"Training samples: {X1_train.shape[0]}")
print(f"Test samples:     {X1_test.shape[0]}")

### 1.3 Fit a linear regression

Scikit-learn follows a consistent API:

1. **Instantiate** the model.
2. **Fit** it on training data (`model.fit(X_train, y_train)`).
3. **Predict** on new data (`model.predict(X_test)`).
4. **Evaluate** with a scoring metric.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

model = LinearRegression()
model.fit(X1_train, y_train)

y_pred = model.predict(X1_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Coefficient (slope): {model.coef_[0]:.4f}")
print(f"Intercept:           {model.intercept_:.4f}")
print(f"R² (test):           {r2:.4f}")
print(f"RMSE (test):         {rmse:.4f}")

> **What is R²?**  
> R² (coefficient of determination) measures the fraction of variance
> in the target that the model explains.  
> - R² = 1 → perfect prediction  
> - R² = 0 → no better than predicting the mean  
> - R² < 0 → worse than predicting the mean

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: regression line over data
x_line = np.linspace(X1_test.min(), X1_test.max(), 100).reshape(-1, 1)
axes[0].scatter(X1_test, y_test, s=10, alpha=0.4, label='Test data')
axes[0].plot(x_line, model.predict(x_line), 'r-', linewidth=2, label='Fit')
axes[0].set(xlabel=f'Response to {odorant_names[odorant_idx]}',
            ylabel='y position', title=f'Simple linear regression (R²={r2:.3f})')
axes[0].legend()

# Right: predicted vs actual
axes[1].scatter(y_test, y_pred, s=10, alpha=0.4)
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
axes[1].plot(lims, lims, 'k--', linewidth=1, label='Perfect prediction')
axes[1].set(xlabel='Actual y position', ylabel='Predicted y position',
            title='Predicted vs. Actual')
axes[1].legend()

plt.tight_layout()
plt.show()

### 1.4 Does the choice of odorant matter?

Some odorants may carry more spatial information than others.  Let's
fit a simple regression for *every* odorant (filtering to its
responders each time) and rank them by R².

In [ ]:
r2_per_odorant = []
n_responders = []

for i in range(X.shape[1]):
    mask = X[:, i] > 0
    if mask.sum() < 10:  # skip odorants with too few responders
        r2_per_odorant.append(np.nan)
        n_responders.append(mask.sum())
        continue
    Xi = X[mask, i].reshape(-1, 1)
    yi = y_pos[mask]
    Xi_train, Xi_test, yi_train, yi_test = train_test_split(
        Xi, yi, test_size=0.2, random_state=42
    )
    m = LinearRegression().fit(Xi_train, yi_train)
    r2_per_odorant.append(r2_score(yi_test, m.predict(Xi_test)))
    n_responders.append(mask.sum())

r2_series = pd.Series(r2_per_odorant, index=odorant_names)
r2_sorted = r2_series.dropna().sort_values(ascending=False)

print("Top 10 odorants by R² (single-feature, responders only):")
for name in r2_sorted.head(10).index:
    idx = list(odorant_names).index(name)
    print(f"  {name:40s}  R²={r2_sorted[name]:.4f}  (n={n_responders[idx]})")
print(f"\nWorst R²: {r2_sorted.iloc[-1]:.4f} ({r2_sorted.index[-1]})")

In [ ]:
plt.figure(figsize=(12, 4))
plt.bar(range(len(r2_sorted)), r2_sorted.values, color='steelblue')
plt.axhline(0, color='gray', linewidth=0.5)
plt.xticks(range(len(r2_sorted)), r2_sorted.index, rotation=90, fontsize=7)
plt.ylabel('R² (test set)')
plt.title('Predictive power of each odorant for y position')
plt.tight_layout()
plt.show()

### Reflection

A single odorant captures only a sliver of the spatial information.
Some odorants are slightly predictive, others are not.  This makes
biological sense — each odorant activates a limited set of receptors
and, by extension, a limited set of glomeruli.

**Can we do better by combining information from multiple odorants?**

---
## 2. Two features, single target

We now use responses to **two odorants** as features:

$$\hat{y} = w_1 x_1 + w_2 x_2 + b$$

This is *multiple* (or multivariate) linear regression — still one
target, but more than one feature.  The model fits a **plane** through
the data instead of a line.

### 2.1 Choose two odorants

We keep **butyric acid** from Section 1 and search for the odorant
that best *complements* it — the one that most improves R² when
paired with butyric acid.  The best partner is not necessarily the
best solo predictor; what matters is that the second feature captures
spatial information that the first one misses.

In [ ]:
# Rank all odorants by R² on all glomeruli (no responder filtering)
r2_all_glom = []
for i in range(X.shape[1]):
    Xi = X[:, i].reshape(-1, 1)
    Xi_tr, Xi_te, yi_tr, yi_te = train_test_split(
        Xi, y_pos, test_size=0.2, random_state=42
    )
    m = LinearRegression().fit(Xi_tr, yi_tr)
    r2_all_glom.append(r2_score(yi_te, m.predict(Xi_te)))

r2_all_sorted = pd.Series(r2_all_glom, index=odorant_names).sort_values(ascending=False)

# Keep butyric acid (best single predictor on all glomeruli)
best_idx = odorant_idx  # from Section 1
best_name = odorant_names[best_idx]

# Find the odorant that best complements it (highest joint R²)
best_r2_pair = -np.inf
second_idx = None
for i in range(X.shape[1]):
    if i == best_idx:
        continue
    X_pair = X[:, [best_idx, i]]
    Xp_tr, Xp_te, yp_tr, yp_te = train_test_split(
        X_pair, y_pos, test_size=0.2, random_state=42
    )
    m = LinearRegression().fit(Xp_tr, yp_tr)
    r2_pair = r2_score(yp_te, m.predict(Xp_te))
    if r2_pair > best_r2_pair:
        best_r2_pair = r2_pair
        second_idx = i

second_name = odorant_names[second_idx]

print(f"Feature 1: {odorants.loc[best_idx, 'Odorant']}  (solo R²={r2_all_glom[best_idx]:.4f})")
print(f"Feature 2: {odorants.loc[second_idx, 'Odorant']}  (solo R²={r2_all_glom[second_idx]:.4f})")
print(f"Pair R²:   {best_r2_pair:.4f}")

In [ ]:
X2 = X[:, [best_idx, second_idx]]

plt.figure(figsize=(6, 5))
sc = plt.scatter(X2[:, 0], X2[:, 1], c=y_pos, s=10, cmap='coolwarm', alpha=0.6)
plt.xlabel(f'Response to {best_name}')
plt.ylabel(f'Response to {second_name}')
plt.title('Two-odorant feature space (color = y position)')
plt.colorbar(sc, label='y position')
plt.show()

### 2.2 Fit and evaluate

The scikit-learn API is exactly the same — the only change is that
`X` now has two columns instead of one.

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y_pos, test_size=0.2, random_state=42
)

model2 = LinearRegression()
model2.fit(X2_train, y2_train)

y2_pred = model2.predict(X2_test)
r2_two = r2_score(y2_test, y2_pred)
rmse_two = np.sqrt(mean_squared_error(y2_test, y2_pred))

print(f"Coefficients: {model2.coef_}")
print(f"Intercept:    {model2.intercept_:.4f}")
print(f"R² (test):    {r2_two:.4f}")
print(f"RMSE (test):  {rmse_two:.4f}")

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(y2_test, y2_pred, s=10, alpha=0.4)
lims = [min(y2_test.min(), y2_pred.min()), max(y2_test.max(), y2_pred.max())]
plt.plot(lims, lims, 'k--', linewidth=1, label='Perfect prediction')
plt.xlabel('Actual y position')
plt.ylabel('Predicted y position')
plt.title(f'Two-feature regression (R²={r2_two:.3f})')
plt.legend()
plt.show()

### 2.3 How does performance grow with more features?

Let's add odorants one at a time (in order of their single-feature
ranking) and see how R² improves.  Here we use all glomeruli so the
comparison is fair across feature counts.

In [ ]:
ranked_indices = [list(odorant_names).index(n) for n in r2_all_sorted.index]

r2_curve = []
for k in range(1, len(ranked_indices) + 1):
    cols = ranked_indices[:k]
    Xk = X[:, cols]
    Xk_tr, Xk_te, yk_tr, yk_te = train_test_split(
        Xk, y_pos, test_size=0.2, random_state=42
    )
    mk = LinearRegression().fit(Xk_tr, yk_tr)
    r2_curve.append(r2_score(yk_te, mk.predict(Xk_te)))

plt.figure(figsize=(7, 4))
plt.plot(range(1, len(r2_curve) + 1), r2_curve, '.-')
plt.xlabel('Number of features (odorants, ranked by single-feature R²)')
plt.ylabel('R² (test set)')
plt.title('Diminishing returns: R² vs. number of features')
plt.tight_layout()
plt.show()

### Reflection

Adding a second odorant can improve the fit, but each additional
feature contributes less and less.  Notice also that with many
features, performance can *decrease* on the test set — a sign of
**overfitting**.  When the model has too many free parameters relative
to the number of samples, it starts fitting noise rather than signal.

This motivates two ideas we will explore next:
- Using **all** features at once, but with **regularization** to
  combat overfitting.
- Predicting **both** x and y simultaneously (**multi-target regression**).

---
## 3. Multi-target, multi-feature regression

Now we use **all 59 odorants** as features and predict **both x and y
coordinates** simultaneously.  The model learns a mapping:

$$[\hat{x}, \hat{y}] = \mathbf{X} \mathbf{W} + \mathbf{b}$$

where **W** is a (59 × 2) weight matrix and **b** is a (2,) bias
vector.

With the full odorant panel as features, we no longer need per-odorant
filtering — every glomerulus has a meaningful 59-dimensional response
profile even if many individual entries are zero.  The pattern of
*which* odorants a glomerulus does and does not respond to is itself
informative.

### 3.1 Prepare multi-target data

In [ ]:
Y = np.column_stack([x_pos, y_pos])  # shape: (n_glomeruli, 2)
print(f"Features X:  {X.shape}")
print(f"Targets  Y:  {Y.shape}  (columns: x, y)")

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

### 3.2 Ordinary least-squares (baseline)

Scikit-learn's `LinearRegression` natively supports multiple targets —
just pass a 2-D `y` array.  It fits an independent regression for
each target.

In [ ]:
model_ols = LinearRegression()
model_ols.fit(X_train, Y_train)

Y_pred_ols = model_ols.predict(X_test)

r2_x_ols = r2_score(Y_test[:, 0], Y_pred_ols[:, 0])
r2_y_ols = r2_score(Y_test[:, 1], Y_pred_ols[:, 1])

print(f"OLS  — R² for x: {r2_x_ols:.4f},  R² for y: {r2_y_ols:.4f}")

With 59 features and a modest number of samples, ordinary
least-squares may be overfitting.  **Regularization** penalizes large
coefficients to keep the model simpler.

### 3.3 Ridge regression (L2 regularization)

Ridge regression adds a penalty proportional to the *sum of squared
coefficients*:

$$\text{Loss} = \sum_i (y_i - \hat{y}_i)^2 + \alpha \sum_j w_j^2$$

The hyperparameter $\alpha$ controls the strength of the penalty.
Larger $\alpha$ → stronger shrinkage → simpler model.

In [ ]:
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X_train, Y_train)

Y_pred_ridge = model_ridge.predict(X_test)

r2_x_ridge = r2_score(Y_test[:, 0], Y_pred_ridge[:, 0])
r2_y_ridge = r2_score(Y_test[:, 1], Y_pred_ridge[:, 1])

print(f"Ridge — R² for x: {r2_x_ridge:.4f},  R² for y: {r2_y_ridge:.4f}")

### 3.4 Choosing alpha with cross-validation

How do we pick the best $\alpha$?  **Cross-validation** systematically
evaluates different values by repeatedly splitting the *training* set
into folds:

1. Hold out one fold as a validation set.
2. Train on the remaining folds.
3. Score on the held-out fold.
4. Average scores across all folds.

`RidgeCV` does this automatically.

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = np.logspace(-2, 4, 50)

# RidgeCV with multi-target: we fit on y-position column first to pick alpha,
# then apply that alpha to the full multi-target problem.
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train, Y_train[:, 1])  # tune on y target

best_alpha = ridge_cv.alpha_
print(f"Best alpha (via 5-fold CV): {best_alpha:.4f}")

# Refit with the chosen alpha on both targets
model_best = Ridge(alpha=best_alpha)
model_best.fit(X_train, Y_train)

Y_pred_best = model_best.predict(X_test)

r2_x_best = r2_score(Y_test[:, 0], Y_pred_best[:, 0])
r2_y_best = r2_score(Y_test[:, 1], Y_pred_best[:, 1])

print(f"Best  — R² for x: {r2_x_best:.4f},  R² for y: {r2_y_best:.4f}")

### 3.5 Compare all models

All rows below use the same sample (all glomeruli, no filtering)
so the R² values are directly comparable.

In [ ]:
results = pd.DataFrame({
    'Model': [
        f'Single odorant — {best_name}',
        f'Two odorants — {best_name} + {second_name}',
        'All 59 odorants — OLS',
        'All 59 odorants — Ridge (α=1)',
        f'All 59 odorants — Ridge (α={best_alpha:.2f})',
    ],
    'R² (y)': [
        r2_all_glom[best_idx],
        r2_two,
        r2_y_ols,
        r2_y_ridge,
        r2_y_best,
    ],
    'R² (x)': [
        np.nan,
        np.nan,
        r2_x_ols,
        r2_x_ridge,
        r2_x_best,
    ],
})
results

### 3.6 Visualize predictions

Let's plot the predicted vs. actual positions for our best model on
the test set.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# Predicted vs actual — x
axes[0].scatter(Y_test[:, 0], Y_pred_best[:, 0], s=10, alpha=0.4)
lims = [Y_test[:, 0].min(), Y_test[:, 0].max()]
axes[0].plot(lims, lims, 'k--', linewidth=1)
axes[0].set(xlabel='Actual x', ylabel='Predicted x',
            title=f'x position (R²={r2_x_best:.3f})')

# Predicted vs actual — y
axes[1].scatter(Y_test[:, 1], Y_pred_best[:, 1], s=10, alpha=0.4)
lims = [Y_test[:, 1].min(), Y_test[:, 1].max()]
axes[1].plot(lims, lims, 'k--', linewidth=1)
axes[1].set(xlabel='Actual y', ylabel='Predicted y',
            title=f'y position (R²={r2_y_best:.3f})')

# Spatial map: actual vs predicted
axes[2].scatter(Y_test[:, 0], Y_test[:, 1], s=12, alpha=0.4, label='Actual')
axes[2].scatter(Y_pred_best[:, 0], Y_pred_best[:, 1], s=12, alpha=0.4,
                marker='x', label='Predicted')
# Draw lines connecting actual → predicted
for i in range(len(Y_test)):
    axes[2].plot([Y_test[i, 0], Y_pred_best[i, 0]],
                 [Y_test[i, 1], Y_pred_best[i, 1]],
                 'gray', linewidth=0.3, alpha=0.3)
axes[2].set(xlabel='x position', ylabel='y position',
            title='Actual vs. predicted locations')
axes[2].legend()

plt.tight_layout()
plt.show()

### 3.7 Which odorants matter most?

The Ridge coefficients tell us how much each odorant contributes to
predicting each target.  Larger absolute coefficients indicate
stronger influence (though correlated features share weight, so
interpret with care).

In [ ]:
coef_df = pd.DataFrame(
    model_best.coef_.T,
    index=odorant_names,
    columns=['x coef', 'y coef'],
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, col in zip(axes, ['x coef', 'y coef']):
    sorted_coefs = coef_df[col].sort_values()
    colors = ['steelblue' if v >= 0 else 'salmon' for v in sorted_coefs]
    ax.barh(range(len(sorted_coefs)), sorted_coefs.values, color=colors)
    ax.set_yticks(range(len(sorted_coefs)))
    ax.set_yticklabels(sorted_coefs.index, fontsize=6)
    ax.set_xlabel('Coefficient')
    ax.set_title(f'Ridge coefficients — {col}')
    ax.axvline(0, color='gray', linewidth=0.5)

plt.tight_layout()
plt.show()

---
## Exercises

1. **Lasso regression:** Replace `Ridge` with `Lasso` (L1 penalty).
   How many coefficients are driven to exactly zero?  What does this
   tell you about feature selection?

2. **High concentration:** Re-run the analysis using the
   high-concentration data (`high` instead of `low`).  Does
   prediction improve?

3. **Non-linear models:** Try `sklearn.ensemble.RandomForestRegressor`
   or `sklearn.neighbors.KNeighborsRegressor` in place of
   `LinearRegression`.  Do non-linear models capture additional
   structure?

4. **Subject effects:** The data pools glomeruli from four animals.
   Train on three subjects and test on the fourth.  Does the spatial
   code generalize across animals?

---
## Summary

| Concept | What we learned |
|---------|----------------|
| Simple linear regression | One feature can capture limited spatial signal |
| Multiple regression | Adding features improves predictions, with diminishing returns |
| Train/test split | Essential to detect overfitting |
| Multi-output regression | Predict both coordinates simultaneously |
| Regularization (Ridge) | Penalty on coefficients prevents overfitting with many features |
| Cross-validation | Principled way to choose hyperparameters |
| Feature importance | Ridge coefficients reveal which odorants drive spatial predictions |